In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas as pd
plt.style.use('ggplot')
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler

import re

pd.set_option('display.max_rows',500)
pd.set_option('display.max_colwidth', 500)

In [2]:
# Uploading datasets

best_cities = pd.read_csv('Best_Cities_for_Startups.csv')
best_countries = pd.read_csv('Best_Countries_for_Startups.csv')
city_pop = pd.read_csv('City_Populations.csv')
country_meta = pd.read_csv('Country_Metadata.csv')

### Exploring data composition - Schema, types, values

> Observing values and types

In [3]:
best_countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 8 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   ranking                       100 non-null    float64
 1   change in position from 2020  100 non-null    object 
 2   country                       100 non-null    object 
 3   total score                   100 non-null    float64
 4   quantity score                100 non-null    float64
 5   quality score                 100 non-null    float64
 6   business score                100 non-null    float64
 7   change in position sign       79 non-null     object 
dtypes: float64(5), object(3)
memory usage: 6.4+ KB


In [4]:
city_pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24664 entries, 0 to 24663
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Country or Area  24664 non-null  object 
 1   Year             24664 non-null  int64  
 2   City             24664 non-null  object 
 3   City type        24664 non-null  object 
 4   Value            24664 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 963.6+ KB


In [5]:
country_meta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265 entries, 0 to 264
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Country Code  265 non-null    object
 1   Region        217 non-null    object
 2   IncomeGroup   216 non-null    object
 3   SpecialNotes  98 non-null     object
 4   TableName     265 non-null    object
dtypes: object(5)
memory usage: 10.5+ KB


In [6]:
best_cities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   position                      1000 non-null   int64  
 1   change in position from 2020  1000 non-null   object 
 2   city                          1000 non-null   object 
 3   country                       1000 non-null   object 
 4   total score                   1000 non-null   float64
 5   quatity score                 1000 non-null   float64
 6   quality score                 1000 non-null   float64
 7   business score                1000 non-null   float64
 8   sign of change in position    798 non-null    object 
dtypes: float64(4), int64(1), object(4)
memory usage: 70.4+ KB


In [7]:
best_cities.describe()

,position,total score,quatity score,quality score,business score
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,500.500000,2.433854,0.343920,1.223350,0.867160
std,288.819436,12.055121,1.347019,10.487355,1.148477
min,1.000000,0.037000,0.010000,0.010000,0.000000
25%,250.750000,0.190750,0.020000,0.030000,0.140000
50%,500.500000,0.367000,0.040000,0.040000,0.270000
75%,750.250000,1.362500,0.180000,0.140000,1.030000
max,1000.000000,328.966000,29.140000,296.020000,3.800000


In [8]:
best_cities.head(20)
best_countries.head(20)
country_meta.head(10)
# country_meta.loc[country_meta['Region'].isna()]

,Country Code,Region,IncomeGroup,SpecialNotes,TableName
0,ABW,Latin America & Caribbean,High income,NaN,Aruba
1,AFE,NaN,NaN,"26 countries, stretching from the Red Sea in the North to the Cape of Good Hope in the South (https://www.worldbank.org/en/region/afr/eastern-and-southern-africa)",Africa Eastern and Southern
2,AFG,South Asia,Low income,Fiscal year end: March 20; reporting period for national accounts data: FY.,Afghanistan
3,AFW,NaN,NaN,"22 countries, stretching from the westernmost point of Africa, across the equator, and partly along the Atlantic Ocean till the Republic of Congo in the South (https://www.worldbank.org/en/region/afr/western-and-central-africa)",Africa Western and Central
4,AGO,Sub-Saharan Africa,Lower middle income,NaN,Angola
5,ALB,Europe & Central Asia,Upper middle income,NaN,Albania
6,AND,Europe & Central Asia,High income,NaN,Andorra
7,ARB,NaN,NaN,Arab World aggregate. Arab World is composed of members of the League of Arab States.,Arab World
8,ARE,Middle East & North Africa,High income,NaN,United Arab Emirates
9,ARG,Latin America & Caribbean,Upper middle income,NaN,Argentina


> Cleaning up column names, eliminating spaces and normalizing formatting

In [22]:
#Replacing spaces with underscores

best_countries.columns = best_countries.columns.str.replace(' ','_')
best_countries.rename({'ranking_':'ranking','change_in_position_sign_':'change_in_position_sign'}, axis=1,inplace=True)

best_countries.columns

Index(['ranking', 'change_in_position_from_2020', 'country', 'total_score',
       'quantity_score', 'quality_score', 'business_score',
       'change_in_position_sign'],
      dtype='object')

In [10]:
best_cities.columns = best_cities.columns.str.replace(' ','_')
best_cities.columns

Index(['position', 'change_in_position_from_2020', 'city', 'country',
       'total_score', 'quatity_score', 'quality_score', 'business_score',
       'sign_of_change_in_position'],
      dtype='object')

In [23]:
##Replacing spaces with underscores and making all strings on column names lower case

city_pop.columns = city_pop.columns.str.replace(' ','_')
city_pop.columns = city_pop.columns.str.lower()
city_pop.columns

Index(['country_or_area', 'year', 'city', 'city_type', 'value'], dtype='object')

### Looking for most improved Countries

In [12]:
# change 'change_in_position_from_2020' column from object to int (non numeric objects need to be filtered out)

best_countries.loc[best_countries['change_in_position_from_2020']=='new  entry']

,ranking,change_in_position_from_2020,country,total_score,quantity_score,quality_score,business_score,change_in_position_sign
98,99.0,new entry,Namibia,0.165,0.04,0.05,0.07,NaN
99,100.0,new entry,Ethiopia,0.162,0.07,0.06,0.03,NaN


In [13]:
#remove rows with non numeric values - not useful - focus on change in ranking position

best_countries=best_countries[best_countries["change_in_position_from_2020"].str.contains(" ")==False]
best_countries.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96 entries, 0 to 97
Data columns (total 8 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   ranking                       96 non-null     float64
 1   change_in_position_from_2020  96 non-null     object 
 2   country                       96 non-null     object 
 3   total_score                   96 non-null     float64
 4   quantity_score                96 non-null     float64
 5   quality_score                 96 non-null     float64
 6   business_score                96 non-null     float64
 7   change_in_position_sign       79 non-null     object 
dtypes: float64(5), object(3)
memory usage: 6.8+ KB


In [14]:
# Change change_in_position_from_2020 column type to int

best_countries["change_in_position_from_2020"]=best_countries["change_in_position_from_2020"].astype('int64')
best_countries.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96 entries, 0 to 97
Data columns (total 8 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   ranking                       96 non-null     float64
 1   change_in_position_from_2020  96 non-null     int64  
 2   country                       96 non-null     object 
 3   total_score                   96 non-null     float64
 4   quantity_score                96 non-null     float64
 5   quality_score                 96 non-null     float64
 6   business_score                96 non-null     float64
 7   change_in_position_sign       79 non-null     object 
dtypes: float64(5), int64(1), object(2)
memory usage: 6.8+ KB


In [15]:
# Removings NaN from best_cities['change_in_position_sign']

best_countries['change_in_position_sign'].fillna('=', inplace=True)
best_countries.head(10)

,ranking,change_in_position_from_2020,country,total_score,quantity_score,quality_score,business_score,change_in_position_sign
0,1.0,0,United States,124.420,19.45,101.17,3.80,=
1,2.0,0,United Kingdom,28.719,8.16,16.86,3.70,=
2,3.0,0,Israel,27.741,5.48,19.14,3.13,=
3,4.0,0,Canada,19.876,6.58,9.75,3.55,=
4,5.0,0,Germany,17.053,3.64,9.93,3.49,=
5,6.0,4,Sweden,15.423,2.40,9.24,3.78,+
6,7.0,7,China,15.128,1.33,11.46,2.34,+
7,8.0,0,Switzerland,14.943,3.82,7.58,3.54,=
8,9.0,2,Australia,13.835,4.46,5.88,3.50,-
9,10.0,6,Singapore,13.745,3.21,7.69,2.84,+


In [16]:
# For all countries that had a negative change in position sign, make the change in position from 2020 a negative numeral

best_countries.loc[best_countries['change_in_position_sign'] == '-', 'change_in_position_from_2020'] = best_countries['change_in_position_from_2020']*-1


In [17]:
best_countries

,ranking,change_in_position_from_2020,country,total_score,quantity_score,quality_score,business_score,change_in_position_sign
0,1.0,0,United States,124.420,19.45,101.17,3.80,=
1,2.0,0,United Kingdom,28.719,8.16,16.86,3.70,=
2,3.0,0,Israel,27.741,5.48,19.14,3.13,=
3,4.0,0,Canada,19.876,6.58,9.75,3.55,=
4,5.0,0,Germany,17.053,3.64,9.93,3.49,=
5,6.0,4,Sweden,15.423,2.40,9.24,3.78,+
6,7.0,7,China,15.128,1.33,11.46,2.34,+
7,8.0,0,Switzerland,14.943,3.82,7.58,3.54,=
8,9.0,-2,Australia,13.835,4.46,5.88,3.50,-
9,10.0,6,Singapore,13.745,3.21,7.69,2.84,+


In [18]:
#Exporting dataset to be used in tableau
best_countries.to_csv('best_countries_v2.csv', sep='\t', index=False)

In [19]:
best_countries['change_in_position_from_2020'].describe()

count    96.000000
mean     -0.343750
std       6.840788
min     -19.000000
25%      -4.000000
50%       0.000000
75%       4.000000
max      18.000000
Name: change_in_position_from_2020, dtype: float64